In [2]:
from time import time
import tensorflow as tf
import multiprocessing as mp
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import CSVLogger, EarlyStopping, LambdaCallback, TensorBoard
import matplotlib.pyplot as plt
import random

Using TensorFlow backend.


In [3]:
def get_model(neurons_in_layer, number_of_layers=2, lr=0.1, activation_function='relu', dropout_rates=0.2):
    """
    Design the model with Multilevel Perceptrons
    :return:
    """
    model = Sequential()

    model.add(Dense(neurons_in_layer, activation=activation_function, input_shape=(3072,)))
    model.add(Dropout(dropout_rates))
    for i in range(number_of_layers-1):
        model.add(Dense(neurons_in_layer, activation=activation_function))
        model.add(Dropout(dropout_rates))

    model.add(Dense(num_classes, activation='softmax'))

    sgd = SGD(lr=lr, decay=1e-6, nesterov=True)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.summary()
    return model

In [4]:
def get_preprocessed_data(x_train, x_test, y_train, y_test):
    """
    Preprocess images. convert them into vector.
    :return:
    """
    x_train = x_train.reshape(50000, 3072)
    x_test = x_test.reshape(10000, 3072)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)

    return x_train, x_test, y_train, y_test

In [5]:
def run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates,
        x_train, x_test, y_train, y_test, callbacks=[]):
    x_train, x_test, y_train, y_test = get_preprocessed_data(x_train, x_test, y_train, y_test)
    model = get_model(neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates)
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)

    scores = model.evaluate(x_test, y_test, verbose=0)
    print(scores)
    return model

In [ ]:
# def run1(epochs, batch_size, model, x_train, x_test, y_train, y_test, callbacks=[]):
#     x_train, x_test, y_train, y_test = get_preprocessed_data(x_train, x_test, y_train, y_test)
#     history = model.fit(x_train, y_train,
#                         batch_size=batch_size,
#                         epochs=epochs,
#                         validation_data=(x_test, y_test),
#                         callbacks=callbacks)

#     scores = model.evaluate(x_test, y_test, verbose=0)
#     print(scores)
#     return model

In [ ]:
# def save_model_prediction_graph(epoch, logs):
#     prediction_probs = model.predict_proba(grid_2d, batch_size=32, verbose=0)
#     plt.figure(figsize=(10,10))
#     sns.set_style("whitegrid")
#     plt.title('Classification with KERAS - epoch: ' + makeIndexOfLength(epoch, 3), fontsize=20)
#     plt.xlabel('X', fontsize=15)
#     plt.ylabel('Y', fontsize=15)
#     plt.contourf(X, Y, prediction_probs.reshape(100, 100), alpha = 0.7, cmap=cm.Spectral)
#     plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train.ravel(), s=50, cmap=plt.cm.Spectral, edgecolors='black')
#     plt.savefig("./" + OUTPUT_DIR + "/keras" + makeIndexOfLength(epoch, 3) + ".png")
#     plt.close()

In [6]:
tensorboard = TensorBoard(log_dir="logs/callbacks/{}".format(time()))

num_classes = 10

if K.backend() == 'tensorflow':
    K.set_image_dim_ordering("th")

core_num = mp.cpu_count()
config = tf.ConfigProto(
    inter_op_parallelism_threads=core_num,
    intra_op_parallelism_threads=core_num)

sess = tf.Session(config=config)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Declare variables
epochs = 15
batch_size = 32

neurons_in_layer = 512
number_of_layers = 2
lr = 0.1
activation_function = 'relu'
dropout_rates = 0.2


x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [ ]:
# # boundary of the graph
# GRID_X_START = -1.5
# GRID_X_END = 1.5
# GRID_Y_START = -1.5
# GRID_Y_END = 1.5
# # output directory (the folder must be created on the drive)
# OUTPUT_DIR = "classification_vizualizations"

# grid = np.mgrid[GRID_X_START:GRID_X_END:100j,GRID_X_START:GRID_Y_END:100j]
# grid_2d = grid.reshape(2, -1).T
# X, Y = grid
# acc_history = []
# loss_history = []
# model = get_model(neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates)
# testmodelcb = LambdaCallback(on_epoch_end=save_model_prediction_graph)
# callbacks = [testmodelcb]
# run1(epochs, batch_size, model, x_train, x_test, y_train, y_test, callbacks)

In [ ]:
csv_logger = CSVLogger('training.log')
callbacks = [csv_logger, tensorboard]
run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates, x_train, x_test, y_train, y_test, callbacks)

In [7]:
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
callbacks = [earlystopper]
run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates, x_train, x_test, y_train, y_test, callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 20s 406us/step - loss: 1

In [8]:
batch_print_callback = LambdaCallback(
    on_batch_begin=lambda batch,logs: print(batch))

callbacks = [batch_print_callback]
epochs = 3
model = run(epochs, batch_size, neurons_in_layer, number_of_layers, lr, activation_function, dropout_rates, x_train, x_test, y_train, y_test, callbacks)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
0
   32/50000 [..............................] - ETA: 8:51 - loss: 2.738

 8704/50000 [====>.........................] - ETA: 21s - loss: 2.2083 - acc: 0.1812272
273
274
275
 8832/50000 [====>.........................] - ETA: 21s - loss: 2.2053 - acc: 0.1823276
277
278
 8928/50000 [====>.........................] - ETA: 21s - loss: 2.2042 - acc: 0.1829279
280
 8992/50000 [====>.........................] - ETA: 21s - loss: 2.2042 - acc: 0.1832281
282
283
284
 9120/50000 [====>.........................] - ETA: 21s - loss: 2.2022 - acc: 0.1839285
286
287
288
 9248/50000 [====>.........................] - ETA: 21s - loss: 2.1994 - acc: 0.1840289
290
291
292
 9376/50000 [====>.........................] - ETA: 21s - loss: 2.2007 - acc: 0.1836293
294
295
296
 9504/50000 [====>.........................] - ETA: 21s - loss: 2.2002 - acc: 0.1839297
298
299
300
 9632/50000 [====>.........................] - ETA: 21s - loss: 2.1981 - acc: 0.1848301
302
303
304
 9760/50000 [====>.........................] - ETA: 21s - loss: 2.1963 - acc: 0.1857305
306
307
308
 9888/50000 

18592/50000 [==========>...................] - ETA: 16s - loss: 2.1123 - acc: 0.2218581
582
583
584
18720/50000 [==========>...................] - ETA: 16s - loss: 2.1108 - acc: 0.2222585
586
587
18816/50000 [==========>...................] - ETA: 15s - loss: 2.1106 - acc: 0.2223588
589
590
18912/50000 [==========>...................] - ETA: 15s - loss: 2.1103 - acc: 0.2223591
592
593
594
19040/50000 [==========>...................] - ETA: 15s - loss: 2.1104 - acc: 0.2225595
596
597
598
19168/50000 [==========>...................] - ETA: 15s - loss: 2.1094 - acc: 0.2228599
600
601
602
19296/50000 [==========>...................] - ETA: 15s - loss: 2.1086 - acc: 0.2231603
604
605
19392/50000 [==========>...................] - ETA: 15s - loss: 2.1076 - acc: 0.2239606
607
608
19488/50000 [==========>...................] - ETA: 15s - loss: 2.1065 - acc: 0.2244609
610
611
19584/50000 [==========>...................] - ETA: 15s - loss: 2.1059 - acc: 0.2247612
613
614
615
19712/50000 [=======

887
888
889
28480/50000 [================>.............] - ETA: 10s - loss: 2.0588 - acc: 0.2453890
891
892
893
28608/50000 [================>.............] - ETA: 10s - loss: 2.0580 - acc: 0.2456894
895
896
28704/50000 [================>.............] - ETA: 10s - loss: 2.0578 - acc: 0.2458897
898
899
28800/50000 [================>.............] - ETA: 10s - loss: 2.0570 - acc: 0.2463900
901
902
903
28928/50000 [================>.............] - ETA: 10s - loss: 2.0552 - acc: 0.2470904
905
906
29024/50000 [================>.............] - ETA: 10s - loss: 2.0546 - acc: 0.2474907
908
909
29120/50000 [================>.............] - ETA: 10s - loss: 2.0535 - acc: 0.2479910
911
912
29216/50000 [================>.............] - ETA: 10s - loss: 2.0534 - acc: 0.2480913
914
915
916
29344/50000 [================>.............] - ETA: 10s - loss: 2.0527 - acc: 0.2482917
918
919
920
29472/50000 [================>.............] - ETA: 10s - loss: 2.0529 - acc: 0.2481921
922
923
924
29600/50

37696/50000 [=====================>........] - ETA: 6s - loss: 2.0199 - acc: 0.26091178
1179
1180
1181
37824/50000 [=====================>........] - ETA: 6s - loss: 2.0200 - acc: 0.26091182
1183
1184
1185
37952/50000 [=====================>........] - ETA: 6s - loss: 2.0198 - acc: 0.26101186
1187
1188
38048/50000 [=====================>........] - ETA: 6s - loss: 2.0194 - acc: 0.26111189
1190
1191
38144/50000 [=====================>........] - ETA: 6s - loss: 2.0191 - acc: 0.26111192
1193
1194
38240/50000 [=====================>........] - ETA: 5s - loss: 2.0187 - acc: 0.26121195
1196
1197
38336/50000 [======================>.......] - ETA: 5s - loss: 2.0188 - acc: 0.26131198
1199
1200
38432/50000 [======================>.......] - ETA: 5s - loss: 2.0184 - acc: 0.26151201
1202
1203
38528/50000 [======================>.......] - ETA: 5s - loss: 2.0178 - acc: 0.26181204
1205
1206
1207
38656/50000 [======================>.......] - ETA: 5s - loss: 2.0173 - acc: 0.26171208
1209
1210
1211


...

43200/50000 [========================>.....] - ETA: 2s - loss: 1.8148 - acc: 0.34101350
1351
1352
1353
43328/50000 [========================>.....] - ETA: 2s - loss: 1.8148 - acc: 0.34111354
1355
1356
1357
43456/50000 [=========================>....] - ETA: 2s - loss: 1.8152 - acc: 0.34111358
1359
1360
1361
43584/50000 [=========================>....] - ETA: 2s - loss: 1.8147 - acc: 0.34141362
1363
1364
1365
43712/50000 [=========================>....] - ETA: 2s - loss: 1.8147 - acc: 0.34151366
1367
1368
1369
43840/50000 [=========================>....] - ETA: 2s - loss: 1.8145 - acc: 0.34161370
1371
1372
1373
43968/50000 [=========================>....] - ETA: 2s - loss: 1.8143 - acc: 0.34181374
1375
1376
1377
44096/50000 [=========================>....] - ETA: 2s - loss: 1.8140 - acc: 0.34201378
1379
1380
1381
44224/50000 [=========================>....] - ETA: 2s - loss: 1.8140 - acc: 0.34191382
1383
1384
1385
44352/50000 [=========================>....] - ETA: 2s - loss: 1.8141 - a

149
150
151
152
 4896/50000 [=>............................] - ETA: 15s - loss: 1.7766 - acc: 0.3652153
154
155
156
157
 5056/50000 [==>...........................] - ETA: 15s - loss: 1.7775 - acc: 0.3641158
159
160
161
162
 5216/50000 [==>...........................] - ETA: 15s - loss: 1.7748 - acc: 0.3645163
164
165
166
167
 5376/50000 [==>...........................] - ETA: 15s - loss: 1.7736 - acc: 0.3646168
169
170
171
172
 5536/50000 [==>...........................] - ETA: 15s - loss: 1.7749 - acc: 0.3643173
174
175
176
177
 5696/50000 [==>...........................] - ETA: 15s - loss: 1.7753 - acc: 0.3643178
179
180
181
182
 5856/50000 [==>...........................] - ETA: 15s - loss: 1.7710 - acc: 0.3663183
184
185
186
187
 6016/50000 [==>...........................] - ETA: 15s - loss: 1.7735 - acc: 0.3649188
189
190
191
192
 6176/50000 [==>...........................] - ETA: 15s - loss: 1.7706 - acc: 0.3651193
194
195
196
 6304/50000 [==>...........................] - ETA: 

16928/50000 [=========>....................] - ETA: 11s - loss: 1.7552 - acc: 0.3676529
530
531
532
533
17088/50000 [=========>....................] - ETA: 11s - loss: 1.7556 - acc: 0.3675534
535
536
537
538
17248/50000 [=========>....................] - ETA: 11s - loss: 1.7563 - acc: 0.3678539
540
541
542
17376/50000 [=========>....................] - ETA: 11s - loss: 1.7568 - acc: 0.3673543
544
545
546
547
17536/50000 [=========>....................] - ETA: 11s - loss: 1.7565 - acc: 0.3676548
549
550
551
552
17696/50000 [=========>....................] - ETA: 11s - loss: 1.7568 - acc: 0.3674553
554
555
556
557
17856/50000 [=========>....................] - ETA: 11s - loss: 1.7571 - acc: 0.3669558
559
560
561
562
18016/50000 [=========>....................] - ETA: 11s - loss: 1.7575 - acc: 0.3671563
564
565
566
567
18176/50000 [=========>....................] - ETA: 11s - loss: 1.7579 - acc: 0.3671568
569
570
571
18304/50000 [=========>....................] - ETA: 11s - loss: 1.7585 -

29152/50000 [================>.............] - ETA: 7s - loss: 1.7533 - acc: 0.3716911
912
913
914
915
29312/50000 [================>.............] - ETA: 7s - loss: 1.7530 - acc: 0.3714916
917
918
919
920
29472/50000 [================>.............] - ETA: 7s - loss: 1.7527 - acc: 0.3718921
922
923
924
925
29632/50000 [================>.............] - ETA: 7s - loss: 1.7525 - acc: 0.3716926
927
928
929
930
29792/50000 [================>.............] - ETA: 7s - loss: 1.7532 - acc: 0.3710931
932
933
934
935
29952/50000 [================>.............] - ETA: 7s - loss: 1.7528 - acc: 0.3713936
937
938
939
30080/50000 [=================>............] - ETA: 7s - loss: 1.7534 - acc: 0.3711940
941
942
943
944
30240/50000 [=================>............] - ETA: 7s - loss: 1.7530 - acc: 0.3713945
946
947
948
30368/50000 [=================>............] - ETA: 7s - loss: 1.7531 - acc: 0.3712949
950
951
952
953
30528/50000 [=================>............] - ETA: 7s - loss: 1.7526 - acc: 0.37

40992/50000 [=======================>......] - ETA: 3s - loss: 1.7497 - acc: 0.37101281
1282
1283
1284
41120/50000 [=======================>......] - ETA: 3s - loss: 1.7495 - acc: 0.37101285
1286
1287
1288
1289
41280/50000 [=======================>......] - ETA: 3s - loss: 1.7493 - acc: 0.37131290
1291
1292
1293
1294
41440/50000 [=======================>......] - ETA: 3s - loss: 1.7490 - acc: 0.37141295
1296
1297
1298
1299
41600/50000 [=======================>......] - ETA: 3s - loss: 1.7488 - acc: 0.37151300
1301
1302
1303
1304
41760/50000 [========================>.....] - ETA: 2s - loss: 1.7490 - acc: 0.37151305
1306
1307
1308
1309
41920/50000 [========================>.....] - ETA: 2s - loss: 1.7489 - acc: 0.37141310
1311
1312
1313
1314
42080/50000 [========================>.....] - ETA: 2s - loss: 1.7487 - acc: 0.37151315
1316
1317
1318
1319
42240/50000 [========================>.....] - ETA: 2s - loss: 1.7481 - acc: 0.37161320
1321
1322
1323
1324
42400/50000 [====================

In [1]:
# cifar10_labels = np.array([
#     'airplane',
#     'automobile',
#     'bird',
#     'cat',
#     'deer',
#     'dog',
#     'frog',
#     'horse',
#     'ship',
#     'truck'])
# def convertCIFER10Data(image):
#     img = image.astype('float32')
#     img /= 255
#     c = np.zeros(32*32*3).reshape((1,32,32,3))
#     c[0] = img
#     return c

# plt.figure(figsize=(16,16))

# right = 0
# mistake = 0

# for i in range(100):
#     index = random.randint(0, x_test.shape[0])
#     image = x_test[index]
# #     data = convertCIFER10Data(image)
#     data = image
# #     plt.subplot(10, 10, i+1)
# #     plt.imshow(image)
# #     plt.axis('off')

#     ret = model.predict(data, batch_size=1) 
#     #print(ret)

#     bestnum = 0.0
#     bestclass = 0
#     for n in [0,1,2,3,4,5,6,7,8,9]:
#         if bestnum < ret[0][n]:
#             bestnum = ret[0][n]
#             bestclass = n

#     if y_test[index] == bestclass:
#         plt.title(cifar10_labels[bestclass])
#         right += 1
#     else:
#         plt.title(cifar10_labels[bestclass] + "!=" + cifar10_labels[y_test[index][0]], color='#ff0000')
#         mistake += 1
                                                                   
# plt.show()
# print("The number of correct answers:", right)
# print("The number of mistake:", mistake)
# print("A correct answer rate:", right/(mistake + right)*100, '%')
